# Instructions
1. Download your activity data from Garmin connect as a csv file at https://connect.garmin.com/modern/activities
2. Upload the .csv file next to this notebook file (i.e. they should be in the same folder/directory together)
3. Select `Kernel > Restart & Run All`
4. Make your selections and whatnot

In [ ]:
%pip install pandas && pip install plotly

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import datetime
import re
import os

from ipywidgets import VBox, HBox, Select, SelectMultiple, Button, Layout, HTML
from typing import Type, Dict


In [ ]:
class DataProcessor:
    
    _ACTIVITY_TYPE_HEADER = "Activity Type"
    _DATE_HEADER = "Date"
    _START_DATE_HEADER = f"Start {_DATE_HEADER}"
    _END_DATE_HEADER = f"End {_DATE_HEADER}"
    _TIME_HEADER = "Time"
    _time_amount_regex = re.compile(
        r"^(?P<hours>[0-9]+):(?P<minutes>[0-5][0-9]):(?P<seconds>[0-5][0-9].?[0-9]?)$"
    )
    
    def __init__(self, file):
        self.file = file
        self.data = pd.read_csv(file)
        
    def get_new_column_headers(self):
        return [
            v for v in self._convert_columns.values()
        ]
        
    def get_weekly_dataframes_by_activity(self, fields=[]):

        def to_minutes(time_str):
            time_dict = self._time_amount_regex.match(time_str).groupdict()
            
            return (
                float(time_dict["hours"]) * 60 
                + float(time_dict["minutes"])
                + float(time_dict["seconds"]) / 60
            )
        
        def datetime_to_date(dt):
            _dt = datetime.datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")
            return datetime.date(year=_dt.year, month=_dt.month, day=_dt.day)

        def get_new_df_row(row, convert_columns):
            df = pd.DataFrame(columns=[v for v in self._convert_columns.values()])
            
            df[self._START_DATE_HEADER] = [row[self._DATE_HEADER]]
            df[self._END_DATE_HEADER] = row[self._DATE_HEADER]
            df[self._ACTIVITY_TYPE_HEADER] = row[self._ACTIVITY_TYPE_HEADER]
            
            for k, v in self._convert_columns.items():
                df[v] = row[k]
                
            return df
        
        def convert_to_float_smart(val):
            if isinstance(val, float):
                ret = val
            elif isinstance(val, int):
                ret = float(val)
            elif isinstance(val, str):
                if ":" in val:
                    val = val.split(":")
                    if len(val) == 2:
                        ret = float(val[0]) + float(val[1])/60
                    elif len(val) == 3:
                        ret = float(val[0]) * 60 + float(val[1]) + float(val[2]) / 60
                    else:
                        raise
                else:
                    ret = float(val.replace(",", "")) if val != "--" else 0
            else:
                raise ValueError(f"how do I convert {val} to a float?")
            return ret
            
        
        assert self._DATE_HEADER not in fields

        self._convert_columns = {field : f"Weekly {field}" for field in fields}
        weekly_dataframes = {}
        
        
        activity_dfs = self.data.groupby(self._ACTIVITY_TYPE_HEADER)
        for k in activity_dfs.groups.keys(): 

            df_group = activity_dfs.get_group(k).sort_values(by=self._DATE_HEADER).reset_index()[
                [self._ACTIVITY_TYPE_HEADER, self._DATE_HEADER] + fields
            ]
            df_group[self._DATE_HEADER] = df_group[self._DATE_HEADER].apply(lambda x: datetime_to_date(x))
            
            # Time field requires special handling
            for field in fields:
                if field == self._TIME_HEADER:
                    df_group[field] = df_group[field].apply(lambda x: to_minutes(x))
                else:
                    df_group[field] = df_group[field].apply(lambda x: convert_to_float_smart(x))

            for i, row in df_group.iterrows():

                if i == 0:
                    weekly_df = get_new_df_row(row=row, convert_columns=self._convert_columns)
                    week_begin = row[self._DATE_HEADER]
                else:
                    if row[self._DATE_HEADER] <= week_begin + datetime.timedelta(days=7):
                        for key, value in self._convert_columns.items():
                            weekly_df.iloc[-1, weekly_df.columns.get_loc(value)] = weekly_df.iloc[-1][value] + row[key]
                        weekly_df.iloc[-1, weekly_df.columns.get_loc(self._END_DATE_HEADER)] = row[self._DATE_HEADER]
                    else:
                        # close the in-process week
                        weekly_df.iloc[
                            -1, weekly_df.columns.get_loc(self._END_DATE_HEADER)
                        ] = weekly_df.iloc[-1][self._START_DATE_HEADER] + datetime.timedelta(days=7)

                        week_begin = week_begin + datetime.timedelta(days=7)

                        # add empty weeks (value of 0) until row is caught up with
                        while week_begin + datetime.timedelta(days=7) < row[self._DATE_HEADER]:
                            _d = {
                                self._ACTIVITY_TYPE_HEADER: [k],
                                self._START_DATE_HEADER: [week_begin],
                                self._END_DATE_HEADER: [week_begin := week_begin + datetime.timedelta(days=7)],
                            }
                            _d.update(
                                {
                                    v: [0] for v in self._convert_columns.values()
                                }
                            )
                            weekly_df = pd.concat(
                                [
                                    weekly_df, 
                                    pd.DataFrame(_d)
                                ],
                                ignore_index=True
                            ).reset_index(drop=True)    
                            
                        new_row_df = get_new_df_row(row=row, convert_columns=self._convert_columns)
                        weekly_df = pd.concat([weekly_df, new_row_df], ignore_index=True).reset_index(drop=True)
                        week_begin = week_begin + datetime.timedelta(days=7)


            weekly_df[self._START_DATE_HEADER] = weekly_df[self._START_DATE_HEADER].astype(str)
            weekly_df[self._END_DATE_HEADER] = weekly_df[self._END_DATE_HEADER].astype(str)
            weekly_dataframes[k] = weekly_df
            
        return weekly_dataframes

In [ ]:
def make_plots(dataframes: Dict[str, Type[pd.DataFrame]], measurements):
    ret = []
    for measurement in measurements:
        ret.append(
            go.FigureWidget(
                data=[
                    go.Scatter(
                        x=df["Start Date"],
                        y=df[measurement],
                        name=k,
                        text=k,
                        mode="lines+markers",
                        connectgaps=False,
                    )
                    for k, df
                    in dataframes.items()
                ],layout=go.Layout(title=f"{measurement} by Activity Type",yaxis={"title": measurement}, xaxis={"title": "Week of"})
            )
        )
        
    return ret

In [ ]:
class UI:
    
    _FILE_SELECT = "file select"
    _COLUMN_SELECT = "column select"
    _PLOT_VIEW = "plot view"
    _ERROR = "error"
    
    def __init__(self):
        # all-encompassing output box
        self._output_box = VBox()
        
        # toolbar
        self._toolbar = HBox(layout=Layout(justify_content="space-between"))
        self._prev_button = Button(description="<<< Prev", button_style="primary")
        self._prev_button.on_click(self._handle_prev_button_click)
        self._next_button = Button(description="Next >>>", button_style="primary")
        self._next_button.on_click(self._handle_next_button_click)
        self._title_box = VBox()
        self._title = HTML("<h2>Garmin Connect Weekly Data Visualizer</h2>")
        self._busy_widget = HTML("<i class='fa fa-spin fa-2x fa-cog'>")
        self._subtitle = HTML()
        self._title_box.children = [self._title, self._subtitle]
        self._toolbar.children = [self._prev_button, self._title_box, self._busy_widget, self._next_button]
        
        # File selection page
        self._file_selector_box = HBox()
        self._file_selector_subtitle = "<h3>Choose your activity file</h3>"
        self._file_selector = Select()
        self._file_selector_label = HTML("<b>Select a File </b>")
        self._file_selector_box.children = [self._file_selector_label, self._file_selector]
        
        # column selection page
        self._column_selector_box = HBox()
        self._column_selector_label = "<h3>Select which columns to plot</h3>"
        self._column_selector_label = HTML("<b>Select Columns to Plot</b>")
        self._column_selector = SelectMultiple(layout=Layout(height="300px"))
        self._column_selector_box.children = [self._column_selector_label, self._column_selector]
        
        # plots page
        self._plot_viewer_box = VBox()
        self._plot_viewer_subtitle = "<h3>View your plots!</h3>"
        self._plot_viewer_label = HTML()
        self._plot_viewer_contents_box = VBox()
        self._plot_viewer_box.children = [self._plot_viewer_label, self._plot_viewer_contents_box]
        
        self._output_box.children=[
            self._toolbar,
            self._file_selector_box,
            self._column_selector_box,
            self._plot_viewer_box,
        ]
        display(self._output_box)
        
    def start(self):
        self._status = self._COLUMN_SELECT
        
        self._handle_prev_button_click(e=None)
        
    def _handle_prev_button_click(self, e):
    
        self._prev_button.disabled = True
        self._next_button.disabled = True

        self._busy_widget.layout.display = "block"
        
        if self._status == self._FILE_SELECT:
            raise AttributeError("this shouldn't be possible. Prev button should be disabled")
            
        elif self._status == self._COLUMN_SELECT:

            self._file_selector_box.layout.display = "block"
            self._column_selector_box.layout.display = "none"
            self._plot_viewer_box.layout.display = "none"

            self._file_selector.options = [f for f in os.listdir() if f.endswith(".csv")]

            self._next_button.disabled = False
            
            self._status = self._FILE_SELECT
            
        elif self._status == self._PLOT_VIEW:
            
            self._file_selector_box.layout.display = "none"
            self._column_selector_box.layout.display = "block"
            self._plot_viewer_box.layout.display = "none"
            
            try:
                df = pd.read_csv(self._file_selector.value)
            except Exception as e:
                self.handle_exception(e)
            
            self._column_selector.options = [
                c for c in df.columns if (
                    c not in {"Date", "Time", "Activity Type", "Favorite"}
                    and "Avg" not in c
                    and "Pace" not in c
                )
            ]
            
            self._prev_button.disabled = False
            self._next_button.disabled = False
            
            self._status = self._COLUMN_SELECT
            
        elif self._status == self._ERROR:
            raise
        else:
            raise
        
        self._busy_widget.layout.display = "none"
        
    def _handle_next_button_click(self, e):
        
        self._prev_button.disabled = True
        self._next_button.disabled = True
        self._busy_widget.layout.display = "block"
        
        if self._status == self._FILE_SELECT:
            
            self._file_selector_box.layout.display = "none"
            self._column_selector_box.layout.display = "block"
            self._plot_viewer_box.layout.display = "none"
            
            try:
                df = pd.read_csv(self._file_selector.value)
            except Exception as e:
                self.handle_exception(e)
            
            self._column_selector.options = [
                c for c in df.columns if (
                    c not in {"Date", "Time", "Activity Type", "Favorite", "Title", "Decompression"}
                    and "Avg" not in c
                    and "Pace" not in c
                )
            ]
            
            self._prev_button.disabled = False
            self._next_button.disabled = False
            
            self._status = self._COLUMN_SELECT           
            
        elif self._status == self._COLUMN_SELECT:
            self._file_selector_box.layout.display = "none"
            self._column_selector_box.layout.display = "none"
            self._plot_viewer_box.layout.display = "block"
            
            self._data_obj = DataProcessor(file=self._file_selector.value)
            weekly_dfs = self._data_obj.get_weekly_dataframes_by_activity(fields=list(self._column_selector.value))
            self._plot_viewer_contents_box.children = make_plots(dataframes=weekly_dfs, measurements=self._data_obj.get_new_column_headers())
            
            self._next_button.disabled = True
            self._prev_button.disabled = False
            
            self._status = self._PLOT_VIEW
            
            
        elif self._status == self._PLOT_VIEW:
            raise AttributeError("this shouldn't be possible. Prev button should be disabled")
            
        elif self._status == self._ERROR:
            raise
        else:
            raise
               
        
        self._busy_widget.layout.display = "none"
                                       
    def handle_exception(self, exception):
        raise exception

In [ ]:
u = UI()
u.start()

